In [1]:
import dryml
from dryml import ObjectDef
import numpy as np

# DRYML Tutorial 4

## DRYML Contexts 2

We've had some experience with contexts, but now we'll discuss how we can avoid creating a context in our current process. This allows us to possibly change how resources are distributed depending on the model. The `dryml.compute_context` decorator generator is provided by DRYML which gives a wrapped method the power to inspect existing compute contexts, or launch itself in a new process with an appropriate context. This makes it easy to interleave code requiring a context with manager code which may require running a variety of models that could have conflicting context requirements.

Now, `dryml.compute_context` is actually a decorator generator meaning, you need to call it to create the decorator you want to use. This allows the user to customize how a given method gets wrapped, and customizes how compute contexts are spawned. DRYML also provides the `dryml.compute` decorator which is just a shortcut to `compute_context()` when generic behavior is fine.

`compute_context` has a couple of important arguments which can be specified when the decorator is created (when calling `compute_context`), and can be overridden when actually calling the function.
* `ctx_context_reqs`: Probably the most important, specifies a specific set of `context_reqs` to use when checking for an existing context or launching a new context. Override at call time with `call_context_reqs`.
* `ctx_use_existing_context` (Default `True`): When `True`, DRYML should try to use an existing context if available. If the existing context doesn't satisfy the given requirements, it will raise a `WrongContextError` exception rather than create a new context. Override at call time with `call_use_existing_context`
* `ctx_dont_create_context` (Default `False`): When `False`, DRYML won't try to create a new context ever. if no context exists, it'll throw a `NoContextError` exception, and if the existing context doesn't satisfy the given requirements, it will raise a `WrongContextError` exception. Override at call time with `call_dont_create_context`
* `ctx_update_objs` (Default `False`): When `True`, DRYML will update objects in the current process with the state of corresponding objects in the remove process upon completion. Override at call time with `call_update_objs`
* `ctx_verbose` (Default `False`): When `True`, DRYML will print some diagnostic information about the whole compute procedure. Override at call time with `call_verbose`.


First, we'll create a function to create our mnist dataset which will first check if an appropriate context is in place. Then we'll write a function for training a model and we'll use the `compute_context` decorator to indicate that this method needs a compute context. Then we'll create three simple models using `sklearn`, `tensorflow`, and `pytorch`. We will then explore the statistical variations of these models by training multiple copies of them and measuring their accuracies.

One complication we'll need to overcome, is that by default, DRYML uses the `spawn` method for creating new subprocesses. This means an entirely new python process is created, and any function definitions defined in this Jupyter notebook will not be available. Typically the way to solve this is to create a python module and place our needed functions in there, however we don't want our users to open an external editor just for the sake of one method. So we've come up with a workaround here you can take with you and use when needed in other contexts!

In [2]:
# Create function to generate datasets
def gen_dataset():
    # import some names
    import dryml
    import tensorflow_datasets as tfds
    from dryml.data.tf import TFDataset

    # Check that the context has tensorflow ability, but don't get specific.
    dryml.context.context_check({'tf': {}})

    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True)
    
    train_ds = TFDataset(
        ds_train,
        supervised=True,
    )
    
    test_ds = TFDataset(
        ds_test,
        supervised=True,
    )
    
    return train_ds, test_ds

class temp_mod_file(object):
    def __init__(self, name: str):
        self.name = name
        self.file = open(self.name, mode='w')

    def write_obj_source(self, obj):
        import inspect
        obj_source = inspect.getsource(obj)
        self.file.write(obj_source)
        self.file.write("\n")
        self.file.flush()

    def __del__(self):
        self.file.close()
        import os
        os.remove(self.name)
        del self.file

# Delete original file
try:
    del mod_file
except NameError:
    pass
mod_file = temp_mod_file('temp_mod.py')
mod_file.write_obj_source(gen_dataset)
del gen_dataset

In [3]:
# Create function to train a model.
# We use ctx_update_objs=True to indicate any objects we give the method should be updated with their
# state at the end of the method.
@dryml.compute_context(ctx_update_objs=True)
def train_model(model):
    from temp_mod import gen_dataset
    train_ds, _ = gen_dataset()
    
    model.prep_train()
    model.train(train_ds)


# Create function to test model
# Since this method doesn't change the models, we don't have to update them after calling it.
@dryml.compute
def test_model(model):
    from dryml.metrics import categorical_accuracy
    from temp_mod import gen_dataset
    _, test_ds = gen_dataset()
    
    model.prep_eval()
    return categorical_accuracy(model, test_ds)

## Create ML Models

Now we'll create a few model classes using `ObjectDef`s. We'll then use `ObjectDef.build` to create instances of these models.

In [4]:
import dryml.models
import dryml.data
import dryml.models.sklearn
import sklearn.neighbors

/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Let's define some common processing steps so we don't have to build full definitions for them every time.
flatten_def = ObjectDef(dryml.data.transforms.Flatten)
best_cat_def = ObjectDef(dryml.data.transforms.BestCat)

In [6]:
# First, we'll build an sklearn model.
sklearn_mdl_def = ObjectDef(
    dryml.models.Pipe,
    flatten_def,
    ObjectDef(
        dryml.models.sklearn.Trainable,
        model=ObjectDef(
            dryml.models.sklearn.ClassifierModel,
            sklearn.neighbors.KNeighborsClassifier,
            n_neighbors=10,
        ),
        train_fn=ObjectDef(
            dryml.models.sklearn.BasicTraining,
            num_examples=500,
            shuffle=True,
            shuffle_buffer_size=5000,
        )
    ),
    best_cat_def,
)

In [7]:
# Now, we can generate, train and test a model.
simple_tf_reqs = {'tf': {}}
temp_model = sklearn_mdl_def.build()
train_model(temp_model, call_context_reqs=simple_tf_reqs)
test_model(temp_model, call_context_reqs=simple_tf_reqs)

2023-02-21 14:04:28.612979: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0.7984775641025641

In [8]:
def train_multiple(model_def=None, num_to_train=None, ctx_reqs=None):
    models = []
    accuracies = []
    for i in range(num_to_train):
        new_model = model_def.build()
        train_model(new_model, call_context_reqs=ctx_reqs)
        acc = test_model(new_model, call_context_reqs=ctx_reqs)
        accuracies.append(acc)
        models.append(new_model)

    return models, accuracies

In [9]:
num_to_train = 5

Now, let's write a function which takes a definition, trains some number of models, tests them and returns the trained models as well as the mean accuracy and accuracy deviation.

In [10]:
sklearn_models, sklearn_accuracies = train_multiple(
    model_def=sklearn_mdl_def,
    num_to_train=num_to_train,
    ctx_reqs={'tf': {}})

2023-02-21 14:04:54.729195: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21

In [11]:
# compute accuracy mean/stddev
print(f"sklearn accuracy: {np.mean(sklearn_accuracies)}+/-{np.std(sklearn_accuracies)}")

sklearn accuracy: 0.8140424679487179+/-0.006648287446539246


Now, let's build a tensorflow model

In [12]:
import tensorflow as tf
import dryml.models.tf

In [13]:
mdl_def = ObjectDef(
    dryml.models.tf.keras.SequentialFunctionalModel,
    input_shape=(28, 28, 1),
    layer_defs=[
        ['Conv2D', {'filters': 16, 'kernel_size': 3, 'activation': 'relu'}],
        ['Conv2D', {'filters': 16, 'kernel_size': 3, 'activation': 'relu'}],
        ['Flatten', {}],
        ['Dense', {'units': 10, 'activation': 'linear'}],
    ]
)
tf_mdl_def = ObjectDef(
    dryml.models.Pipe,
    ObjectDef(
        dryml.models.tf.keras.Trainable,
        model=mdl_def,
        train_fn=ObjectDef(
            dryml.models.tf.keras.BasicTraining,
            epochs=2
        ),
        optimizer=ObjectDef(
            dryml.models.tf.Wrapper,
            tf.keras.optimizers.Adam,
        ),
        loss=ObjectDef(
            dryml.models.tf.Wrapper,
            tf.keras.losses.SparseCategoricalCrossentropy,
            from_logits=True,
        )
    ),
    ObjectDef(
        dryml.data.transforms.BestCat
    )
)

In [14]:
tf_models, tf_accuracies = train_multiple(
    model_def=tf_mdl_def,
    num_to_train=num_to_train,
    ctx_reqs={'tf': {'gpu/0': 1.}}
)

2023-02-21 14:08:30.481477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
2023-02-21 14:08:40.773108: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


Epoch 1/2
1500/1500 [==============================] - 6s 2ms/step - loss: 0.2948 - val_loss: 0.0981
Epoch 2/2
1500/1500 [==============================] - 4s 2ms/step - loss: 0.0649 - val_loss: 0.1093


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:08:57.301482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:09:06.357473: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> dev

Epoch 1/2
1500/1500 [==============================] - 6s 3ms/step - loss: 0.4539 - val_loss: 0.1257
Epoch 2/2
1500/1500 [==============================] - 4s 2ms/step - loss: 0.0783 - val_loss: 0.1218


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:09:32.978154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:09:42.121351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> dev

...

/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:10:08.353131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:10:17.495015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13834 MB memory:  -> dev

Epoch 1/2
1500/1500 [==============================] - 6s 2ms/step - loss: 0.4095 - val_loss: 0.0967
Epoch 2/2
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0643 - val_loss: 0.1195


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:10:43.746834: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13836 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:10:52.908733: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13830 MB memory:  -> dev

Epoch 1/2
1500/1500 [==============================] - 6s 3ms/step - loss: 0.4298 - val_loss: 0.1132
Epoch 2/2
1500/1500 [==============================] - 4s 2ms/step - loss: 0.0726 - val_loss: 0.1720


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-02-21 14:11:19.369676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13835 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0004:05:00.0, compute capability: 7.0
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
# compute accuracy mean/stddev
print(f"tf accuracy: {np.mean(tf_accuracies)}+/-{np.std(tf_accuracies)}")

tf accuracy: 0.9713141025641026+/-0.0048250221850399426


And now, let's have a look at a similar pytorch model, We'll have to add another step to change the order of the indicies of the data since pytorch expects data in nchw format while tensorflow uses nhwc format. We'll also have to add a `TorchDevice` transformation to make sure the data is on the cpu.

In [16]:
import dryml.models.torch
import dryml.data.torch
import torch

/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
mdl_def = ObjectDef(
    dryml.models.torch.generic.Sequential,
    layer_defs=[
        [torch.nn.LazyConv2d, (16, 3), {}],
        [torch.nn.ReLU, (), {}],
        [torch.nn.LazyConv2d, (16, 3), {}],
        [torch.nn.ReLU, (), {}],
        [torch.nn.Flatten, (), {}],
        [torch.nn.LazyLinear, (10,), {}],
    ]
)
torch_mdl_def = ObjectDef(
    dryml.models.Pipe,
    ObjectDef(
        dryml.data.transforms.Transpose,
        axes=(2, 0, 1)
    ),
    ObjectDef(
        dryml.data.transforms.Cast,
        dtype='float32'
    ),
    ObjectDef(
        dryml.models.torch.generic.Trainable,
        model=mdl_def,
        train_fn=ObjectDef(
            dryml.models.torch.generic.BasicTraining,
            optimizer=ObjectDef(
                dryml.models.torch.generic.TorchOptimizer,
                torch.optim.Adam,
                mdl_def,
            ),
            loss=ObjectDef(
                dryml.models.torch.base.Wrapper,
                torch.nn.CrossEntropyLoss
            )
        )
    ),
    ObjectDef(
        dryml.data.torch.transforms.TorchDevice,
        device='cpu'
    ),
    ObjectDef(
        dryml.data.transforms.BestCat
    )
)

In [20]:
torch_models, torch_accuracies = train_multiple( model_def=torch_mdl_def,
    num_to_train=num_to_train,
    ctx_reqs={'tf': {}, 'torch': {'gpu/0': 1.}}
)

/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|██████████| 1875/1875 [00:19<00:00, 95.00it/s, loss=0.0103] 


Epoch 1 - Average Loss: 0.010318919583297975


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy mod

Epoch 1 - Average Loss: 0.007857729579828446


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy mod

...

/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy mod

Epoch 1 - Average Loss: 0.007870576143279323


/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy mod

Epoch 1 - Average Loss: 0.008460258307691159



/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mkrafcz2/HAL_Projects/DRYML/venv_dryml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [21]:
# compute accuracy mean/stddev
print(f"torch accuracy: {np.mean(torch_accuracies)}+/-{np.std(torch_accuracies)}")

torch accuracy: 0.9697315705128207+/-0.0020773536595646304
